### spare code
useful at the time, but not any more, may be useful in the future

In [1]:
# so if we have an image with multiple things inside, we just want the biggest crystal -> want to return the index of these, so can just use it to extract it
# Convert the dataframe to a binary image where 0 is True (activated) and all else is False
binary_image = (one_crystal == 0)

# Label the connected components
labeled_image, num_features = label(binary_image)

plt.imshow(binary_image, cmap='gray')

props = regionprops(labeled_image)
if props:
    # Find the component with the maximum area
    largest_component = max(props, key=lambda x: x.area)
    print(largest_component)

    # Extract x-values (column indices) of the largest component
    coords = largest_component.coords
    x_values = np.unique(coords[:, 1])  # column indices are the x-values
else:
    x_values = np.array([])  # No components found

NameError: name 'one_crystal' is not defined

In [ ]:
## functions for particle calculations
## Min + Max dimensions calculation - Feret Diameters
# found other thing worked better

def calculate_feret_diameters(points):
    """Calculate both minimum and maximum Feret diameters using the convex hull and rotating calipers."""
    # Check if all points are collinear
    if np.linalg.matrix_rank(points - points[0]) < 2:
        # Calculate maximum and minimum distances between points as they are on a line
        dists = distance_matrix(points, points)
        max_feret = np.max(dists)
        min_feret = np.min(dists[dists > 0])  # Exclude zero distance (self to self)
        return min_feret, max_feret
        
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    
    min_diameter = np.inf  # Start with infinitely large diameter
    max_diameter = 0  # Start with zero diameter
    num_vertices = len(hull_points)

    for i in range(num_vertices):
        j = (i + 1) % num_vertices
        p1, p2 = hull_points[i], hull_points[j]
        # Vector from p1 to p2
        edge_vec = p2 - p1
        edge_length = np.linalg.norm(edge_vec)
        max_diameter = max(max_diameter, edge_length)  # Maximum Feret diameter
        
        # Normalize the edge vector
        edge_vec /= edge_length
        # Project points onto the line normal to the edge
        normal_vec = np.array([-edge_vec[1], edge_vec[0]])
        projections = np.dot(hull_points - p1, normal_vec)
        min_diameter = min(min_diameter, max(projections) - min(projections))  # Minimum Feret diameter
    
    return min_diameter, max_diameter

def feret_diameters(contours):
    if contours:
        # Assume the largest contour is the main one (if there are multiple)
        largest_contour = max(contours, key=lambda x: x.shape[0])
        # Using convex hull to calculate Feret diameters
        hull = ConvexHull(largest_contour)
        hull_points = largest_contour[hull.vertices]
    
        # Calculate max Feret diameter
        dists = distance_matrix(hull_points, hull_points)
        max_feret = np.max(dists)
    
        # Calculate min Feret diameter
        min_feret = np.inf
        num_vertices = len(hull_points)
        for i in range(num_vertices):
            j = (i + 1) % num_vertices
            p1, p2 = hull_points[i], hull_points[j]
            edge_vec = p2 - p1
            edge_length = np.linalg.norm(edge_vec)
            edge_vec /= edge_length
            normal_vec = np.array([-edge_vec[1], edge_vec[0]])
            projections = np.dot(hull_points - p1, normal_vec)
            min_feret = min(min_feret, max(projections) - min(projections))

        return min_feret, max_feret

def calculate_feret_diameter(coords):
    """Calculate the Feret diameter for a set of points."""
    dist_matrix = distance_matrix(coords, coords)
    feret_diameter = np.max(dist_matrix)
    return feret_diameter

### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def calculate_properties(crystal):
    dat = {}
    #thresh = threshold(crystal.astype('uint8'), 0, 255, THRESH_BINARY + THRESH_OTSU)

    # Optionally fill holes
    thresh = binary_fill_holes(crystal).astype(int)
    contours = measure.find_contours(thresh, 0.5)
    # Sort contours by area (largest first) and select the largest contour
    contour = max(contours, key=lambda x: x.shape[0])
    
    # Label the image based on the threshold
    labeled_image = measure.label(thresh)
    
    # Extract properties using regionprops
    region = measure.regionprops(labeled_image)[0]  # Assumes largest labeled region corresponds to largest contour
    
    # Store the extracted properties in the data structure
    dat['len'] = region.major_axis_length
    dat['wid'] = region.minor_axis_length
    dat['area'] = region.area
    dat['orientation'] = region.orientation
    dat['centroid'] = region.centroid

    return dat

In [ ]:
## old when extracting data from individual crystals
## this was within the loop
multi[particle.label] = [coords[0][0],coords[-1][0],s_idx,e_idx] # y1,y2,x1,x2 
                # x values are our index also needed for extracting time
                # i.e. [f2ds['ImageData'][y1:y2,x1:x2]


keys = list(multi.keys())
extract = f2ds['ImageData'][multi[keys[0]][0]-2:multi[keys[0]][1]+3,multi[keys[0]][2] -2:multi[keys[0]][3]+3] ## added 2 pixel buffer around single crystal
start_t = time_xr['utc_time'][multi[keys[0]][2]]
end_t = time_xr['utc_time'][multi[keys[0]][3]]

In [ ]:
# possibly 2 images are captured simultaneously, so will have to capture this in the name when extracting i.e. _0 or _1.h5
# this is a function to extract the image data + save appropriately

def extract_save_image(x, save_location, int='0'):
    im_s_idx = time_xr['pix_sum'][x].values
    im_e_idx = time_xr['pix_sum'][x+1].values

    one_crystal = ds_image[:,im_s_idx:im_e_idx]
    
    crystal_str = str(time_xr['utc_time'][im_s_idx].values)
    individual_file = f'{crystal_str[5:7]}{crystal_str[8:10]}-{crystal_str[11:13]}{crystal_str[14:16]}{crystal_str[17:19]}{crystal_str[20:22]}_{int}.h5'

    save_name = save_location+individual_file

    print(save_name)
    
    with h5py.File(save_name, 'w') as f:
    # Create a dataset in the HDF5 file
        f.create_dataset('data', data=one_crystal)


for i in range(100):
    if i = 0:
        extract_save_image
    elif time_xr['utc_time'][i] == time_xr['utc_time'][i-1]:
        # this account for repeat in time for two seperate images
        extract_save_image(i,save_loc,int='1')

    else:
        extract_save_image(i,save_loc)

In [6]:
## extracting particles where area is > than specif threshold
## taking our cleaned data, where the files are >4 in length

# 1. want to find the objects in the sections
# 2. identify objects in which the max dimension is > than specific threshold
# 3. extract start + end index of said particle
# 4. use this index to slice + get corresponding UTC time associated with particle

area_threshold = 30 # need this minimum number of pixels to be classified
pixel_resolution = 10 # mu

binary_image = (one_crystal == 0) ## important, convert regions where 0 = True (our bits of interest), all else false

particle_data = []

# Label the connected components
labeled_image, num_features = label(binary_image) # identify connected true areas
# labeled_image = array, with each true area given a number to identify them
# num_features = number of unique connected components in image. Have to literally have adjacent pixel, not diagonal (this will make them seperate)

props = regionprops(labeled_image) # creates quick list of properties describing each feature detected in the image.
# has: label, area, centroid, bounding box - for each feature

## only doing this if anything there i.e. if props
## want to extract only particles that surpass area threshold 
if props:
    multi = {}
    for particle in props:
        if particle.area >= area_threshold:

            ## basic info
            coords = particle.coords # basically gives coords of each point of interest
            x_values = np.unique(coords[:, 1])
            s_idx = int(selected_values[i] + x_values[0])
            e_idx = int(selected_values[i] + x_values[-1])
            
            ## more complex stats
            spec_region = stats_description(particle)


            ######## TRUNCATION CHECKING ###################
                
            min_row, min_col, max_row, max_col = spec_region.bbox

            '''
            print(min_row)
            print(min_col)
            print(filled_part.shape[0])
            print(filled_part.shape[1])
            '''

            #print(spec_region.convex_area)
            #print(spec_region.area)
            
            ## convex area
            convex_threshold = 0.7
            
            if spec_region.convex_area > 0:
                area_ratio = spec_region.area / spec_region.convex_area
                if area_ratio < convex_threshold:  # threshold can be set, e.g., 0.9 or 0.8
                    print("Particle might be truncated.")

            #########


            print(spec_region.eccentricity)
            print(spec_region.solidity)
            # solidity = Ratio of pixels in the region to pixels of the convex hull image 
            # centroid location? 
            
            # nice way of saving data
            particle_data.append({
                    #"image_index": image_index,
                    "particle_label": particle.label,
                    "start_index": s_idx,
                    "end_index": e_idx,
                    "start_time": time_xr['utc_time'][s_idx],  # assuming 'time_xr' is pre-defined and syncs with indices
                    "end_time": time_xr['utc_time'][e_idx],
                    "major_axis_length": spec_region.major_axis_length * pixel_resolution,
                    "minor_axis_length": spec_region.minor_axis_length * pixel_resolution,
                    "orientation": spec_region.orientation,
                    "centroid": spec_region.centroid,
                    "area": (spec_region.area * (pixel_resolution**2)),
                    "perimeter": (spec_region.perimeter * pixel_resolution),
                    "y0": coords[0][0],
                    "y1": coords[-1][0]
                })
            
    particle_df = pd.DataFrame(particle_data)